In [2]:
import torch
import torch.nn as nn 
from transformers import BlipProcessor, BlipForConditionalGeneration, BertTokenizer, BertForQuestionAnswering
from PIL import Image

In [3]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

img_path = "maxresdefault.jpg"
results = model("maxresdefault.jpg")
detected_objects = results.pandas().xyxy[0]['name'].tolist()
print("Detected Objects:", detected_objects)

Using cache found in /home/naren/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-11 Python-3.12.4 torch-2.4.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/home/naren/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Detected Objects: ['horse']


In [4]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


img = Image.open(img_path)
inputs = processor(img, return_tensors="pt")
outputs = model.generate(**inputs)

# Decode the output
caption = processor.decode(outputs[0], skip_special_tokens=True)
print("Generated Caption:", caption)

Generated Caption: a black horse running through a field


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Example Query
query = input("What is that you want to know about the image")

# Tokenize the inputs (caption + query)
inputs = tokenizer.encode_plus(query, caption, return_tensors="pt")

# Get the answer span
start_scores, end_scores = model(**inputs, return_dict=False)
start_idx = torch.argmax(start_scores)
end_idx = torch.argmax(end_scores) + 1

# Decode the answer
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx]))
print("Answer:", answer)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Answer: a black horse
